[too many cells documantation](https://gregoryschwartz.github.io/too-many-cells/#orgbadfe10)

Objective:
- Create functional `genes.tsv` file.
- Create functional `barcodes.tsv` file.

We have a matrix in sparse format, which reportesnts matrix that have:
| File        | Savings    | Savings                               |
| --------    | -------    | -------                               |
| matrix.mtx  | raw count  | The coordinates and values (x,y,count)|
| barcodes.tsv| "cells"    | The names for the columns (y-axis)    |
| genes.tsv   | "genes"    | The names for the rows (x-axis)       |
- `kmer` aka `barcodes` as columns.
- `trimer` aka `genes` as rows.

If we want to create labels for each cell (20-mer) -> we need a to create `labels.csv` in the following format:
|       item          |    label  |
| ------------------- | --------- | 
| AAACCTGCAGTAACGG-1  | Marrow    | 
| AAACGGGAGACCGGAT-1  | Marrow    | 
| AAACGGGAGCGCTCCA-1  | Marrow    | 



<b> Tasks </b>
1. Ceate a proper `genes.tsv` file (rows) -> from the `5_{}_{}_filtered_trimers_VarRemain.csv` file.
2. Create a proper `barcodes.tsv` file (columns) -> from the `"3_{}_{}_VarRemain.csv"` file.
3. Ceate a `labels.csv` file that connects the `barcodes.tsv` to specific label (metadata).

In [ ]:
# Packages import
import os
import pandas as pd

# Matrix creation data
temp_data_folder = os.path.join("temp_data","covid_vaccine_new")
file_genes = "5_covid_vaccine_new_7_filtered_trimers_VarRemain.csv"
file_barcodes = "3_covid_vaccine_new_7_VarRemain.csv"

# Loading matrix creation data to python
genes_input = pd.read_csv(os.path.join(temp_data_folder, file_genes), index_col=0)
barcodes_input = pd.read_csv(os.path.join(temp_data_folder, file_barcodes), index_col=0)

# Current barocdes and genes files
input_data_folder = os.path.join("C:/","Users","Daniel","Desktop","tms_preprocessing","input_data","covid_vaccine_new")
genes = "%5Ccovid_vaccine_new_7_genes.tsv"
barcodes = "%5Ccovid_vaccine_new_7_barcodes.tsv"

# Loading matrix current data to python
genes = pd.read_csv(os.path.join(input_data_folder, genes), sep="\t", index_col=0)
barcodes = pd.read_csv(os.path.join(input_data_folder, barcodes), sep="\t", index_col=0)

In [ ]:
seqK = pd.read_csv(os.path.join(temp_data_folder, "1_covid_vaccine_new_7_seqK.csv")).reset_index(names="id")
kmers = barcodes_input
merged = pd.merge(left=seqK, right=kmers, how="right", on="id")

In [ ]:
from source.helpers import mysql_qry, read_json
import pandas as pd
import os

config = read_json()

def test_04_create_labels():
    # Importing the metadata table from the MySQL serever
    metadata_label = config["database"]["metadata_label"]
    metadata_qry = "SELECT * FROM covid_vaccine_new.sample_metadata"
    metadata_df = mysql_qry(qry=metadata_qry)

    # Verifing that there is `metadata_label` input in the `sql_congif.json` file
    if metadata_label == "":
        print("> No metadata label selected, aborting creation of `labels.csv`")
    
    else:
        print(f"metadata label `{metadata_label}` selected, creating `labels.csv`.")

        # Dictionary that link sample_id to metadata label:
        filt_df = metadata_df[metadata_df["key"] == metadata_label]
        labels_dict = {int(i):j for i,j in zip(filt_df.sample_id.values, filt_df.value.values)}

        # Loading the required datasets in order backtrack to the first metadata contining files
        """
        Steps of the backtracking:
        1. getting the `id` labels from the k-mers used to create the barcodes (f03).
        2. merging the filterd k-mers (f03) with the k-mer table (f01) by 'id'.

        """
        
        #### ! revise paths for final class method ! ####
        temp_folder = os.path.join("temp_data","covid_vaccine_new")
        tms_folder = os.path.join("tms_input","covid_vaccine_new")
        database = "covid_vaccine_new"
        subject_id = "7"
        
        # Importing the preprocsssed tables
        f01_seqk = os.path.join(temp_folder, "1_{}_{}_seqK.csv".format(database, subject_id))
        f03_var_remain = os.path.join(temp_folder, "3_{}_{}_VarRemain.csv".format(database, subject_id))
        barcodes = os.path.join(tms_folder, "covid_vaccine_new_7_barcodes.tsv".format(database, subject_id))

        # Joining the tables to get the sample id
        f01_df = pd.read_csv(f01_seqk).reset_index(names="id")
        f03_df = pd.read_csv(f03_var_remain)
        barcodes_df = pd.read_csv(barcodes, sep="\t", index_col=None, header=None)
        barcodes_df.columns = ["kmer"]

        # Using the labels_dict to map the labels (via sample id)
        merged_df = pd.merge(left=f01_df, right=f03_df, how="right", on="id")[["kmer","id","sample_id"]]
        merged_df["label"] = merged_df.sample_id.map(labels_dict)

        # Adding labels to the k-kmers
        labels_df = pd.merge(left=barcodes_df, right=merged_df, on="kmer", how="left")[["kmer", "label"]]
        labels_df.columns = [["item","label"]]
        labels_path = os.path.join(tms_folder, "{}_{}_labels.csv").format(database, subject_id)
        labels_df.to_csv(labels_path, index=0)

        print(f"{labels_path} labels file created.")

test = test_04_create_labels()

> Established connecntion to the covid_vaccine_new database.
> Query executed successfully: 
SELECT * FROM covid_vaccine_new.sample_metadata
> MySQL connenction terminated.
metadata label `cell_subset` selected, creating `labels.csv`.


---- 
* [V] Need to update the sql custom function.
* [V] Need to ceate another function to link wanted metadata to the table by barcodes.
* [X] Need to cheeck why f06 have 2 less rows than itps input f04.
* [X] Need to run the tms on the server.